In [9]:
# 03_modelado.ipynb
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score



In [10]:
# 1. Cargar datos procesados
X_train = pd.read_csv('../data/X_train.csv')
X_test = pd.read_csv('../data/X_test.csv')
y_train = pd.read_csv('../data/y_train.csv').values.ravel()
y_test = pd.read_csv('../data/y_test.csv').values.ravel()



In [11]:
# 2. Definir Modelos a Probar [cite: 232]
modelos = {
    "Regresión Logística": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42)
}

mejor_modelo = None
mejor_auc = 0


In [12]:
# 3. Entrenamiento y Comparación
print("RESULTADOS DE EVALUACIÓN:\n")

for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    y_prob = modelo.predict_proba(X_test)[:, 1] # Probabilidad para curva ROC
    
    # Métricas [cite: 238]
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    
    print(f"--- {nombre} ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"AUC-ROC: {auc:.4f}")
    print(classification_report(y_test, y_pred))
    print("-" * 30)
    
    # Guardar el ganador
    if auc > mejor_auc:
        mejor_auc = auc
        mejor_modelo = modelo
        nombre_ganador = nombre


RESULTADOS DE EVALUACIÓN:

--- Regresión Logística ---
Accuracy: 0.8600
AUC-ROC: 0.7646
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       229
           1       0.84      0.51      0.63        71

    accuracy                           0.86       300
   macro avg       0.85      0.74      0.77       300
weighted avg       0.86      0.86      0.85       300

------------------------------
--- Random Forest ---
Accuracy: 0.8567
AUC-ROC: 0.7599
              precision    recall  f1-score   support

           0       0.86      0.97      0.91       229
           1       0.82      0.51      0.63        71

    accuracy                           0.86       300
   macro avg       0.84      0.74      0.77       300
weighted avg       0.85      0.86      0.84       300

------------------------------
--- XGBoost ---
Accuracy: 0.8433
AUC-ROC: 0.7315
              precision    recall  f1-score   support

           0       0.87      0.93    

In [13]:
# 4. Guardar el Mejor Modelo
print(f"✅ El mejor modelo fue: {nombre_ganador} con AUC: {mejor_auc:.4f}")

# Guardamos el archivo .pkl en la carpeta de la App Web
ruta_modelo = '../web_app/credit_risk/ml_models/modelo_riesgo_v1.pkl'
joblib.dump(mejor_modelo, ruta_modelo)

print(f"Modelo guardado exitosamente en: {ruta_modelo}")

✅ El mejor modelo fue: Regresión Logística con AUC: 0.7646
Modelo guardado exitosamente en: ../web_app/credit_risk/ml_models/modelo_riesgo_v1.pkl
